In [1]:
from pathlib import Path
import os

base_dir = Path(os.getcwd())
scrape_out_path = "data"
result_path = "output/movie_rankings.csv"
result_cleaned_path = "output/movie_rankings_cleaned.csv"
vis_path = "output/visual"
vis_genre_cluster_path = "output/visual/1_genre_cluster.png"
vis_rel_turnover_path = "output/visual/2_budget_grossing_ratio.png"
vis_budget_comp_path = "output/visual/3_budget_comparison.png"
vis_lucrative_path = "output/visual/4_genre_lucrative.png"
vis_lucrative_mean_path = "output/visual/5_genre_mean_lucrative.png"
vis_lucrative_mean_year_path = "output/visual/6_year_mean_lucrative.png"                                                

In [2]:
import scrapy

class BoxOfficeSpider(scrapy.Spider):
    name = "boxoffice"
    start_urls = [
        "https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/",
        "https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?offset=200",
        "https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?offset=400"
        ]
    
    def parse(self, response):
        # Chat-GPT: for-loop and rank
        for row in response.css("tr"):
            rank = row.css("td:nth-child(1)::text").get()
            title = row.css("td.a-text-left.mojo-field-type-title a.a-link-normal::text").get()
            url = row.css("td.a-text-left.mojo-field-type-title a.a-link-normal::attr(href)").get()
            print(rank)
            if title and url and rank:
                yield {
                    'Rank': rank.strip(),
                    'Title': title.strip(),
                    'Url': response.urljoin(url.strip()),
                }

In [ ]:
import subprocess
import sys
from pathlib import Path


def run_spider():
    base_dir = Path().resolve()
    data_dir = base_dir / "data"
    output_path = data_dir / "movie_links.json"
    spider_path = base_dir / "scrapy_spider.py"
    
    data_dir.mkdir(parents=True, exist_ok=True)

    # Chat-GPT 14 - 15
    if output_path.exists():
        output_path.unlink()
    
    print("Running Scrapy spider...")
    
    spider_code = """
import scrapy

class BoxOfficeSpider(scrapy.Spider):
    name = "boxoffice"
    start_urls = [
        "https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/",
        "https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?offset=200",
        "https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?offset=400"
        ]
    
    def parse(self, response):
        # Chat-GPT: for-loop and rank
        for row in response.css("tr"):
            rank = row.css("td:nth-child(1)::text").get()
            title = row.css("td.a-text-left.mojo-field-type-title a.a-link-normal::text").get()
            url = row.css("td.a-text-left.mojo-field-type-title a.a-link-normal::attr(href)").get()

            if title and url and rank:
                yield {
                    'Rank': rank.strip(),
                    'Title': title.strip(),
                    'Url': response.urljoin(url.strip()),
                }
"""
    if not spider_path.exists():
        spider_path.write_text(spider_code)
    
    # Chat-GPT 15 - 21 (runs and waits for subprocess)
    result = subprocess.run(
        [sys.executable, "-m", "scrapy", "runspider", "scrapy_spider.py", "-o", str(output_path)],
        cwd=str(base_dir),
        capture_output=True,
        text=True
    )

    if result.returncode != 0:
        print("Spider failed:")
        print(result.stderr)
        return False

    print("\nSpider finished successfully.\n")  
    print(f"Output path: {output_path} \n")
    
    return True


In [26]:
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


class BoxOfficeScraper:
    def __init__(self, max_movies):
        self.max_movies = max_movies

    def scrape_budget_genre_from_webpaths(self, webpaths):
        budgets = []
        # Chat-GPT 16-17
        with open(webpaths, encoding='utf-8') as f:
            movies = json.load(f)

        # Chat-GPT 20 (opens browser controlled by selenium)
        driver = webdriver.Chrome()
        try:
            for movie in movies[:self.max_movies]:
                rank = movie["Rank"]
                title = movie["Title"]
                url = movie["Url"]
                # Chat-GPT 27 (navigates to url in the browser)
                driver.get(url)

                try:
                    # Chat-GPT 29 - 36 (waits for element to appear)
                    budget_span = WebDriverWait(driver, 1).until(
                        # (expects condition, waits for element given to appear in DOM, \
                        # using XPATH cause we cant access via ID)
                        EC.presence_of_element_located((By.XPATH, "//span[text()='Budget']/following-sibling::span"))
                    )
                    budget = budget_span.text
                except:
                    budget = "N/A"
                    
                try:
                    genre_span = WebDriverWait(driver, 1).until(
                        EC.presence_of_element_located((By.XPATH, "//span[text()='Genres']/following-sibling::span"))
                    )
                    genre = genre_span.text    
                except:
                    genre = "N/A"
                    

                print(f"Rank: {rank}, Titel: {title}, Budget: {budget}, Genre: {genre}")
                budgets.append({"Rank": rank, "Title": title, "Budget": budget, "Genre": genre})
              
        finally:
            driver.quit()
        
        return pd.DataFrame(budgets)
    
    
        
    def scrape_grossings_as_df(self):
        max = int(self.max_movies/3)
        url = "https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/"
        df1 = pd.read_html(url, header=0)[0][:max]
        url2 = "https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?offset=200" 
        df2 = pd.read_html(url2, header=0)[0][:max]
        url3 = "https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?offset=400" 
        df3 = pd.read_html(url3, header=0)[0][:max]
        combined_df = pd.concat([df1, df2, df3], ignore_index=True)
        return combined_df
            

In [31]:
from pathlib import Path
import pandas as pd


class MovieDataProcessor:

    @staticmethod
    def merge_data(grossings_csv, budgets_csv):
        if not Path(grossings_csv).exists() or not Path(budgets_csv).exists():
            raise FileNotFoundError("One or both input CSV files were not found.")
        
        df1 = pd.read_csv(grossings_csv)
        df2 = pd.read_csv(budgets_csv)         
        
        df1 = df1.sort_values("Rank")
        df2 = df2.sort_values("Rank")
        
        df_combined = pd.merge(df1, df2, on='Rank', how='left')
        return df_combined

    @staticmethod   
    def export_to_csv(df, filepath):
        # df.set_index("Rank", inplace=True)
        df.to_csv(filepath)
    
    @staticmethod    
    def get_entries_for_column(df, column_name):
        return df[column_name].unique().tolist()
    
    @staticmethod
    def get_budget_chunks_difference(df, budget, chunk_size, chunks):
        
        df['Budget diff Mean'] = 0.0
        df['Chunks'] = ""
        for i in range(1, chunks + 1):
            section_start = chunk_size * (i-1)
            section_end = min(chunk_size * i, len(df))
            mean = df[budget][section_start:section_end].mean()
            df.loc[section_start:section_end - 1, 'Budget diff Mean'] = mean
            df.loc[section_start:section_end - 1, 'Chunks'] = f"Chunk {i}"
        return df
    
    @staticmethod
    def get_percent_difference_columns(df, budget, grossing):
        df['Percent'] = round((((df[grossing] - df[budget]) / df[budget]) * 100).astype(float), 2)
        return df
    
    @staticmethod
    def get_total_difference_columns(df, budget, grossing):
        df['Total Gross Diff'] = df[grossing] - df[budget]
        return df
    
    @staticmethod
    def get_total_difference_mean_per_genre(df, genre, diff):
        genre_means = df.groupby(genre)[diff].mean()
        df['Total Gross Diff Mean'] = df[genre].map(genre_means)
        return df
    
    @staticmethod
    def get_total_difference_mean_per_year(df, year):
        year_means = df.groupby(year)['Total Gross Diff'].mean()
        df['Total Gross Diff Mean'] = df[year].map(year_means)
        return df
        
    @staticmethod
    def count_genre_occurring_per_year(df, genre, year):
        df['Parent Genre'] = df[genre].apply(lambda x: " ".join(x.split()[:2]))
        genre_count_by_year = df.groupby([year, 'Parent Genre']).size().reset_index(name='Genre Count')
        df = df.merge(genre_count_by_year, how='left', on=[year, 'Parent Genre'])
        return df
    
    @staticmethod
    def explode_genre(df, genre):
        df[genre] = df[genre].str.split(" ")
        df = df.explode(genre).reset_index(drop=True)
        return df

In [32]:
from pathlib import Path

import matplotlib
from processor import MovieDataProcessor
from matplotlib import patches
import pandas as pd
import matplotlib.pyplot as plt



class BoxOfficeVisualizer:
    def __init__(self, data_path):
        self.genre_dic = {}
        self.data_path = data_path
        
        
    def visualize_genre_cluster(self, output_path, title, x_title, y_title, bar_plt = False, color = True, connect = False):     
        if not Path(self.data_path).exists():
            raise FileNotFoundError("CSV file was not found or path incorrect.")
    
        df = pd.read_csv(self.data_path)
        df_helper = BoxOfficeVisualizer._create_helper_columns(df)


        x_data = df_helper['Budget']
        y_data = df_helper['Worldwide Lifetime Gross']
        df_helper.to_csv("output/test.csv")
        self._setup_plot(df_helper, title, x_title, y_title, x_data, y_data, output_path, bar_plt, color, connect)
        
    def visualize_genre_relative_comparison(self, output_path, title, x_title, y_title, bar_plt = False, color = True, connect = False):
        if not Path(self.data_path).exists():
            raise FileNotFoundError("CSV file was not found or path incorrect.")
    
        df = pd.read_csv(self.data_path)

        df_percent = MovieDataProcessor.get_percent_difference_columns(df, "Budget", "Worldwide Lifetime Gross")
        df_helper = BoxOfficeVisualizer._create_helper_columns(df)
        
        x_data = df_percent['Budget']
        y_data = df_percent['Percent']

        self._setup_plot(df_helper, title, x_title, y_title, x_data, y_data, output_path, bar_plt, color, connect)
        
    def visualize_budget_comparison(self, output_path, title, x_title, y_title, chunk_size, chunks, bar_plt = False, color = True, connect = False):
        if not Path(self.data_path).exists():
            raise FileNotFoundError("CSV file was not found or path incorrect.")
    
        df = pd.read_csv(self.data_path)
        section_end = chunk_size * chunks
        if (section_end > len(df)):
            raise Exception("Wanted Chunk sizes dont match the df")
        
        df_helper = BoxOfficeVisualizer._create_helper_columns(df)
        df_cut = MovieDataProcessor.get_budget_chunks_difference(df_helper, "Budget", chunk_size, chunks)
        
        df_cut = df.iloc[:section_end,].copy()
        x_data = df_cut['Chunks']
        y_data = df_cut['Budget diff Mean']
        self._setup_plot(df, title, x_title, y_title, x_data, y_data, output_path, bar_plt, color, connect)
        
        
    def visualize_genre_most_lucrative(self, output_path, title, x_title, y_title, bar_plt = False, color = True, connect = False):
        if not Path(self.data_path).exists():
            raise FileNotFoundError("CSV file was not found or path incorrect.")
    
        df = pd.read_csv(self.data_path)

        df_total_diff = MovieDataProcessor.get_total_difference_columns(df, "Budget", "Worldwide Lifetime Gross")
        df_helper = BoxOfficeVisualizer._create_helper_columns(df_total_diff)

        x_data = df_helper['First_Two_Letters']
        y_data = df_helper['Total Gross Diff']
        
        self._setup_plot(df_helper, title, x_title, y_title, x_data, y_data, output_path, bar_plt, color, connect)
        
        
    def visualize_genre_most_lucrative_mean(self, output_path, title, x_title, y_title, bar_plt = False, color = True, connect = False):
        if not Path(self.data_path).exists():
            raise FileNotFoundError("CSV file was not found or path incorrect.")
    
        df = pd.read_csv(self.data_path)
        
        df_helper = BoxOfficeVisualizer._create_helper_columns(df)
        df_helper = MovieDataProcessor.get_total_difference_columns(df_helper, "Budget", "Worldwide Lifetime Gross")
        df_helper = MovieDataProcessor.get_total_difference_mean_per_genre(df_helper, "First_Two_Letters", "Total Gross Diff")

        x_data = df_helper['First_Two_Letters']
        y_data = df_helper['Total Gross Diff Mean'] 
        
        self._setup_plot(df_helper, title, x_title, y_title, x_data, y_data, output_path, bar_plt, color, connect)
        
        
    def visualize_year_most_lucrative_mean(self, output_path, title, x_title, y_title, bar_plt = False, color = True, connect = False):
        if not Path(self.data_path).exists():
               raise FileNotFoundError("CSV file was not found or path incorrect.")
        
        df = pd.read_csv(self.data_path)
            
        df_helper = BoxOfficeVisualizer._create_helper_columns(df)
        df_total_diff = MovieDataProcessor.get_total_difference_columns(df_helper, "Budget", "Worldwide Lifetime Gross")
        df_diff_mean = MovieDataProcessor.get_total_difference_mean_per_year(df_total_diff, "Year")
        df_diff_mean = df_diff_mean.sort_values(by='Year')

        x_data = df_diff_mean['Year']
        y_data = df_diff_mean['Total Gross Diff Mean']
            
        self._setup_plot(df_diff_mean, title, x_title, y_title, x_data, y_data, output_path, bar_plt, color, connect)

    
    def _setup_plot(self, df, titel, x_titel, y_titel, x_data, y_data, output_path, bar_plt = False, color = True, connect = False):
        plt.figure(figsize=(22, 18))
        if color == True:
            df = self._setup_column_genre_color(df)
            if bar_plt == True:
                plt.bar(x_data, y_data, color=df['Color'])
            else:
                plt.scatter(x_data, y_data, c=df['Color'])
        else:
            if bar_plt == True:
                plt.bar(x_data, y_data, color="grey")
            elif connect == True:
                plt.plot(x_data, y_data, marker='o', linestyle='-', color='steelblue')   
            else: 
                plt.scatter(x_data, y_data)
               
        plt.title(titel)
        plt.xlabel(x_titel)
        plt.ylabel(y_titel)

        if color == True: 
            unique_genres = df['First_Two_Genre'].unique()
            # Chat-GPT  131 - 132
            handles = [patches.Patch(color=self.genre_dic.get(genre, 'gray'), label=genre) for genre in unique_genres]
            plt.legend(handles=handles, title="Genres")
        
        plt.savefig(output_path)

        plt.close()
        
    def _setup_column_genre_color(self, df):  
        genre_list = df['First_Two_Genre'].unique().tolist()
        color_list = BoxOfficeVisualizer._get_colors(len(genre_list))

        for idx, genre in enumerate(genre_list):
            self.genre_dic[genre] = color_list[idx]
            
        df['Color'] = df['First_Two_Genre'].apply(lambda x: self.genre_dic.get(x, 'gray'))
        
        return df
    
    @staticmethod   
    def _get_colors(genre_size, gap = 3):
        # Chat-Gpt from...
        total = genre_size + gap
        cmap = matplotlib.colormaps.get_cmap('hsv')
        color_list = [cmap(i / total) for i in range(genre_size)]
        return color_list
        # ... till
        
    @staticmethod    
    def _create_helper_columns(df):
        df['First_Two_Genre'] = df['Genre'].apply(lambda x: " ".join(sorted(x.split(" ")[0:2])))
        # Chat-GPT 162
        df['First_Two_Letters'] = df['First_Two_Genre'].apply(lambda x: "".join([word[:2] for word in x.split(" ")]))
        return df

In [33]:
from pathlib import Path
import pandas as pd

class Cleaner:
    
    @staticmethod
    def clean_data(data_path, output_path):
        if not Path(data_path).exists():
            raise FileNotFoundError("CSV file was not found or path incorrect.")
        
        df = pd.read_csv(data_path) 

        df = df.dropna()\
            .drop(columns=["Domestic Lifetime Gross", "Domestic %", "Foreign Lifetime Gross", "Foreign %", "Title_y"])\
            .rename(columns={"Title_x": "Title"})\
            .reset_index(drop=True)

        cleaned_df = Cleaner._clean_money_data(df, "Worldwide Lifetime Gross", "Budget")
        print(f"Cleaned money data...")
        
        cleaned_df.to_csv(output_path)
        print(f"Cleaned dataframe saved into: {output_path}")
        
    @staticmethod
    def _clean_money_data(df, *col_names):
        for col_name in col_names:       
            df[col_name] = df[col_name].str.replace("[$,]", "", regex=True).astype(int)
        return df

    @staticmethod   
    def _clean_percent_data(df, *col_names):
        for col_name in col_names:       
            df[col_name] = df[col_name].str.replace("[%]", "", regex=True).astype(float)
        return df

In [37]:
while True:
    choice = input("1: scrape movie links, 2: scrape movie data, 3: build dataframe, 4: clean dataframe, 5: make graph, 6: exit, 7: test | (1, 2, 3, 4, 5, 6, 7): ").strip()   
    try:
        if choice == "1":
            run_spider()
            
        elif choice == "2":
            scraper = BoxOfficeScraper(600)
            budgets_df = scraper.scrape_budget_genre_from_webpaths(f"{scrape_out_path}/movie_links.json")
            budgets_df.to_csv(f"{scrape_out_path}/movie_budgets_and_genres.csv", index=False)
            print(f"Budgets and Genres written into: {scrape_out_path}")
            
            grossing_df = scraper.scrape_grossings_as_df()
            grossing_df.to_csv(f"{scrape_out_path}/movie_grossings.csv", index=False)
            print(f"Grossings written into: {scrape_out_path}")

        elif choice == "3":
            movie_rankings = MovieDataProcessor.merge_data(f"{scrape_out_path}/movie_grossings.csv", f"{scrape_out_path}/movie_budgets_and_genres.csv")
            MovieDataProcessor.export_to_csv(movie_rankings, f"{result_path}")
            print(f"Exported combined data to: {result_path}/")
                       
        elif choice == "4":
            Cleaner.clean_data(result_path, result_cleaned_path)
            
        elif choice == "5":
            movie_visual = BoxOfficeVisualizer(result_cleaned_path)
            
            movie_visual.visualize_genre_cluster(vis_genre_cluster_path, "Box Office: Worldwide Gross vs Budget", "Budget ($) ", "Worldwide Lifetime Gross ($)")
            print(f"\nExported visualized of data to: {vis_genre_cluster_path}")
            
            movie_visual.visualize_genre_relative_comparison(vis_rel_turnover_path, "Percentage Increase in Earnings relative to Budget", "Budget ($)", "Percent Increase Budget Return (%)")
            print(f"Exported visualized of data to: {vis_rel_turnover_path}")
            
            movie_visual.visualize_budget_comparison(vis_budget_comp_path, "Show Budget for top 500 Movies in 5 Chunks", "Chunks (100 highest Großing per Chunk descending)", "Mean Budget ($)", 100, 4, bar_plt=True, color=False)
            print(f"Exported visualized of data to: {vis_budget_comp_path}")
            
            movie_visual.visualize_genre_most_lucrative(vis_lucrative_path, "Turnover Per Movie", "Genre", "Turnover ($)")
            print(f"Exported visualized of data to: {vis_lucrative_path}")
            
            movie_visual.visualize_genre_most_lucrative_mean(vis_lucrative_mean_path, "Difference in Turnover Mean per Genre", "Genre", "Mean Turnover ($)", bar_plt=True)
            print(f"Exported visualized of data to: {vis_lucrative_mean_path}")
            
            movie_visual.visualize_year_most_lucrative_mean(vis_lucrative_mean_year_path, "Difference in Turnover Mean per Year", "Year", "Mean Turnover ($)", color=False, connect=True)
            print(f"Exported visualized of data to: {vis_lucrative_mean_year_path}\n")
            
        elif choice == "6":
            print("exiting...\n")
            break
        
        elif choice == "7":
            print()
        
        else:
            print("Invalid input\n")
            
    except Exception as e:
        print(f"\n An error occurred: \n {e} \n")
        break

Exported combined data to: output/movie_rankings.csv/
Cleaned money data...
Cleaned dataframe saved into: output/movie_rankings_cleaned.csv

Exported visualized of data to: output/visual/1_genre_cluster.png
Exported visualized of data to: output/visual/2_budget_grossing_ratio.png
Exported visualized of data to: output/visual/3_budget_comparison.png
Exported visualized of data to: output/visual/4_genre_lucrative.png
Exported visualized of data to: output/visual/5_genre_mean_lucrative.png
Exported visualized of data to: output/visual/6_year_mean_lucrative.png

exiting...

